In [39]:
import json
import datetime
from web3 import Web3
import mysql.connector as mysql

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# init web3
infura_url = secret['INFURAURL1']
web3 = Web3(Web3.HTTPProvider(infura_url))
print(f"Connected to infura: {infura_url}")

Connected to infura: https://mainnet.infura.io/v3/1070d8486bc64afdb9c730d6f2850d6c


In [28]:
# load abi
with open('abi_erc20.json') as f:
    abi = json.load(f)

In [29]:
wallet = '0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a'
token = '0x0954906da0Bf32d5479e25f46056d22f08464cab'

In [30]:
contract = web3.eth.contract(abi=abi, address=Web3.toChecksumAddress(str(token)))

In [31]:
token_symbol = str(contract.functions.symbol().call())
token_name = str(contract.functions.name().call())
print(token_symbol, token_name)

INDEX Index


In [33]:
raw_balance = contract.functions.balanceOf(wallet).call()
balance = raw_balance/10**(contract.functions.decimals().call())
print(f"wallet {wallet} has {balance} tokens")

wallet 0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a has 271.78937788270594 tokens


# bot sql queries

## select guilds

In [45]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT guild_id, guild_name, unite_setup_channel_id, created_at FROM discord_guilds;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
guilds = []
for record in records:
    guilds.append(dict(zip(['guild_id', 'guild_name', 'unite_setup_channel_id', 'created_at'], record)))

## select list of unite setup channels

In [47]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT distinct(unite_setup_channel_id) FROM discord_guilds;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
unite_setup_channels = []
for record in records:
    unite_setup_channels.append(record[0])

## insert guild

In [44]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "INSERT INTO discord_guilds (guild_id, guild_name, unite_setup_channel_id, created_at) VALUES (%s, %s, %s, %s);"
values = (1337, "test_guild", 1234, str(datetime.datetime.now()).split('.')[0])
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record inserted")
cursor.close()
db.close()

1 record inserted


## insert rule

In [79]:
guild_id = 1234
token_address = "0x1234"
token_min = 5
token_max = None
role_id = '795012514324807682'
role_name = "level 1"

In [80]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "INSERT INTO discord_rules (guild_id, token_address, token_min, token_max, role_id, role_name, created_at) VALUES (%s, %s, %s, %s, %s, %s, %s);"
values = (guild_id, token_address, token_min, token_max, role_id, role_name, str(datetime.datetime.now()).split('.')[0])
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record inserted")
cursor.close()
db.close()

1 record inserted


## select rules

In [54]:
guild_id = 1234

In [63]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT token_address, token_min, token_max, role_id, role_name FROM discord_rules where guild_id = {guild_id};'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
rules = []
for record in records:
    rule = dict(zip(['token_address', 'token_min', 'token_max', 'role_id', 'role_name'], record))
    rules.append(rule)

In [64]:
rules

[{'token_address': '0x1234',
  'token_min': 5,
  'token_max': None,
  'role_id': '795012514324807682',
  'role_name': 'level 1'}]

## reset rules

In [81]:
guild_id = 1234

1234

In [83]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "DELETE from discord_rules where guild_id = %s;"
values = (str(guild_id),)
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record deleted")
cursor.close()
db.close()

1 record deleted
